# [Keras + Universal Sentence Encoder = Deep Meter](https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/) 

This notebook creates an autoencoder using the Universal Sentence Encoder. The autoencoder output is CMUdict syllables. The dataset is that subset of Allison Parrish's Project Gutenberg poetry archive which happens to scan in iambic pentameter.

The notebook is based on Chengwei Zhang's example of wrapping the USE inside a larger tensorflow model saves to a Keras model (without save the USE itself in the TF model).

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.

Since there are 10 one-hot values for 10 sets of 6k syllables, this is "multi-label classification"
Changes for multi-label classification:
sigmoid activation instead of softmax
binary_crossentropy

Text format is tab-separated, 2 columns: first text, second multi-level
array of syllables:


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [1]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
%cd /content
!git clone https://github.com/LanceNorskog/deep_meter || true
%cd deep_meter
!git pull
# could not figure out how to read gzipped files as text!
!gunzip -qf blobs/*.gz || true
!gunzip -qf prepped_data/*.gz || true

/content
Cloning into 'deep_meter'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 252 (delta 3), reused 7 (delta 2), pack-reused 243
Receiving objects: 100% (252/252), 20.77 MiB | 7.46 MiB/s, done.
Resolving deltas: 100% (134/134), done.
/content/deep_meter
Already up to date.
gzip: blobs/*.gz: No such file or directory


In [2]:
# boilerplate from base notebook
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Nadam, Adam
np.random.seed(10)

Using TensorFlow backend.


In [0]:
# github deep_meter code
import utils
# should not need this to use utils.flatten but is true anyway?
from itertools import chain
import subprocess
import syllables
# misc for this notebook
from ast import literal_eval

import scipy



In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [5]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.


In [0]:
# read classified poetry lines: text tab [['syll', 'la', 'ble'], ...]
# clip to only most common syllables with syllable manager
def get_data(filename, syll_mgr, num_symbols):
    lines = open(filename, 'r').read().splitlines()
    text_lines = []
    enc_array = []
    num_syllables = syll_mgr.get_size()
    for i in range(0, len(lines)):
      parts = lines[i].split("\t")
      label = utils.flatten(literal_eval(parts[1]))
      if len(label) != num_symbols:
        continue
      enc = np.zeros((num_symbols * syll_mgr.get_size()), dtype=np.int8)
      for j in range(num_symbols):
        enc[num_syllables * j + syll_mgr.get_encoding(label[j])] = 1
      text_lines.append([parts[0]])
      enc_array.append(enc)

    lines = None
    return (np.array(text_lines), np.array(enc_array))


In [0]:
# syllables in descending order of occurrence - 6k in gutenberg.iambic_pentameter, 15k total
# clamp to most common 100 syllables while debugging- use NCE to get all syllables or interesting number
# 98 + pause + wildcard
# iambic pentameter
num_symbols = 10
#syll_mgr = syllables.syllables(num_syllables)
syll_mgr = syllables.syllables()
num_syllables = syll_mgr.get_size() 
syll_weights = {}
counts = syll_mgr.get_counts()
maxim = np.max(counts)
for t in range(10):
  for i in range(len(counts)):
    if counts[i] > 0:
      syll_weights[t * len(counts) + i] = 1/(counts[i]/maxim)
    else:
      syll_weights[t * len(counts) + i] = 0
    

In [8]:
print(syll_weights[15])
(train_text, train_label) = get_data('prepped_data/gutenberg.iambic_pentameter.train', syll_mgr, num_symbols)
num_training = len(train_text)
#train_text = train_text[0:10000]
#train_label = train_label[0:10000]

(test_text, test_label) = get_data('prepped_data/gutenberg.iambic_pentameter.test', syll_mgr, num_symbols)
num_testing = len(test_text)
train_label.shape


11.88058472308009


(58498, 66350)

In [9]:

# slow
num_epochs = 50
adam_lr = 0.001
adam_opt = Adam(lr=adam_lr)
nadam_opt = tf.contrib.opt.NadamOptimizer(adam_lr)
print(type(adam_opt))

<class 'tensorflow.python.keras.optimizers.Adam'>


In [0]:
# weighted binary crossentropy?


## Wrap embed module in a Lambda layer
Explicitly cast the input as a string

In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]


## Assemble model

In [0]:
def create_weighted_binary_crossentropy(zero_weight, one_weight):

    def weighted_binary_crossentropy(y_true, y_pred):

        # Original binary crossentropy (see losses.py):
        # K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

        # Calculate the binary crossentropy
        b_ce = K.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        # Return the mean error
        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy
  
custom_binary_crossentropy = create_weighted_binary_crossentropy(1/(np.sqrt(num_syllables)), 1 - 1/np.sqrt(num_syllables))

In [0]:
POS_WEIGHT=10
def weighted_binary_crossentropy2(target, output):
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(targets=target,
                                                    logits=output,
                                                    pos_weight=POS_WEIGHT)
    return tf.reduce_mean(loss, axis=-1)


In [0]:
def hamming_loss(y_true, y_pred):
  return K.mean(y_true*(1-y_pred)+(1-y_true)*y_pred)

In [24]:
output_activation='sigmoid'
# changed accuracy from 'choose your own accuracy'
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,), name='TF-Hub')(input_text)
embedding = layers.Dropout(0.1)(embedding)
dense = layers.Dense(1024, activation='relu', name='Convoluted')(embedding)
#dense = layers.Dropout(0.5)(dense)
#dense = layers.Dense(2048, activation='relu', name='Medium')(dense)
#dense = layers.Dropout(0.5)(dense)
#dense = layers.Dense(4096, activation='relu', name='Smooth')(dense)
pred = layers.Dense(num_symbols * num_syllables, activation=output_activation, name='Flatout')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss=custom_binary_crossentropy, 
              optimizer=nadam_opt, 
              metrics=['categorical_crossentropy']
             )
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
TF-Hub (Lambda)              (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
Convoluted (Dense)           (None, 1024)              525312    
_________________________________________________________________
Flatout (Dense)              (None, 66350)             68008750  
Total params: 68,534,062
Trainable params: 68,534,062
Non-trainable params: 0
_________________________________________________________________


## Train Keras model and save weights
This only trains and save our Keras layers not the embed module' weights.

In [0]:
use_saved_model=False
if not use_saved_model or not os.path.exists('./model.h5'):
  with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=num_epochs,
            callbacks = [EarlyStopping(patience=5)],
            batch_size=32,
            verbose=2
            ,class_weight=syll_weights
    )
    model.save_weights('./model.h5')


Train on 58498 samples, validate on 4200 samples
Epoch 1/50
 - 154s - loss: 0.0103 - categorical_crossentropy: 85.2406 - val_loss: 3.8945e-04 - val_categorical_crossentropy: 81.4413
Epoch 2/50
 - 150s - loss: 0.0076 - categorical_crossentropy: 79.0229 - val_loss: 3.5140e-04 - val_categorical_crossentropy: 78.3123
Epoch 3/50
 - 150s - loss: 0.0065 - categorical_crossentropy: 74.9549 - val_loss: 3.2851e-04 - val_categorical_crossentropy: 76.2410
Epoch 4/50


In [15]:
!ls -alh | grep model.h5

-rw-r--r-- 1 root root 262M Oct 31 01:49 model.h5


## Make predictions

In [0]:

#new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(test_text, batch_size=32)

In [17]:
first = predicts[0]
max_d = -1000000
min_d = 1000000
max_i = 0
min_i = 0
for i in range(len(predicts) - 1):
  v = predicts[i]
  d = scipy.spatial.distance.cosine(v, first)
  if max_d < d:
    max_d = d
    max_i = i
  if min_d > d:
    min_d = d
    min_i = i
print("Min and max distances: {0} and {1}".format(min_d, max_d))
print(test_text[max_i])
syll_mgr.interpret2(predicts[max_i])

Min and max distances: 9.000109205681284e-07 and 0.6183236302713405
['For he can say to others, _I was there!_']
[7, 26, 26, 26, 26, 26, 26, 26, 26, 26]
['IH N', 'HH IH M', 'HH IH M', 'HH IH M', 'HH IH M', 'HH IH M', 'HH IH M', 'HH IH M', 'HH IH M', 'HH IH M']


In [18]:
for i in range(10,100):
  print(test_text[i])
  syll_mgr.interpret2(predicts[i])
  print('')

['Of humble seeming: and, behind them all,']
[7, 286, 15, 193, 3, 350, 3, 294, 58, 1735]
['IH N', 'M AH NG', 'L IY', 'V EH', 'AE N D', 'M OW S T', 'AE N D', 'G R EY T', 'DH EH M', 'M AY R']

['Behind his head upon the shoulders lay,']
[4, 195, 8, 786, 43, 786, 8, 786, 8, 68]
['AH', 'B AH V', 'HH IH Z', 'SH OW L', 'W IH', 'SH OW L', 'HH IH Z', 'SH OW L', 'HH IH Z', 'L OW']

['Discover countries, with a kindred heart']
[4, 94, 5, 94, 5, 94, 179, 111, 179, 77]
['AH', 'N EY', 'AH V', 'N EY', 'AH V', 'N EY', 'SH AH N Z', 'W ER L D', 'SH AH N Z', 'L AH V']

['Was open in his praise, and plaudits rose']
[21, 213, 8, 228, 26, 113, 25, 113, 8, 26]
['R IH', 'T EH N', 'HH IH Z', 'L AH', 'HH IH M', 'HH AA R T', 'W AA Z', 'HH AA R T', 'HH IH Z', 'HH IH M']

['A spear the hero bore of wondrous strength,']
[4, 502, 4, 616, 100, 3, 2, 616, 1002, 767]
['AH', 'AE K', 'AH', 'S AO R D', 'M ER', 'AE N D', 'DH AH', 'S AO R D', 'L IH S', 'S P IH R']

['Were things indifferent to the Shepherds thoughts.']
[10

In [19]:
categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
print("Categorie: {0}".format(categories))
predict_labels = [categories[logit] for logit in predict_logits]
predict_labels

NameError: ignored

In [0]:

os.remove('./model.h5')